In [1]:
class TicTacToe:
    def __init__(self):
        self.board = [' '] * 9
        self.current_winner = None

    def print_board(self):
        for i in range(0, 9, 3):
            print('| ' + ' | '.join(self.board[i:i+3]) + ' |')

    def available_moves(self):
        return [i for i in range(9) if self.board[i] == ' ']

    def empty_squares(self):
        return ' ' in self.board

    def make_move(self, square, letter):
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.check_winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def check_winner(self, square, letter):
        row = square // 3
        col = square % 3

        # Check row
        if all(self.board[row*3 + i] == letter for i in range(3)):
            return True

        # Check column
        if all(self.board[col + 3*i] == letter for i in range(3)):
            return True

        # Check diagonals
        if square % 2 == 0:
            if all(self.board[i] == letter for i in [0, 4, 8]) or all(self.board[i] == letter for i in [2, 4, 6]):
                return True

        return False


In [3]:
def minimax_alpha_beta(state, player, maximizing_player, alpha=-float('inf'), beta=float('inf')):
    max_player = maximizing_player
    other_player = 'O' if player == 'X' else 'X'

    if state.current_winner == other_player:
        return {'position': None, 'score': 1 * (len(state.available_moves()) + 1) if other_player == max_player else -1 * (len(state.available_moves()) + 1)}
    elif not state.empty_squares():
        return {'position': None, 'score': 0}

    if player == max_player:
        best = {'position': None, 'score': -float('inf')}
    else:
        best = {'position': None, 'score': float('inf')}

    for possible_move in state.available_moves():
        state.make_move(possible_move, player)
        sim_score = minimax_alpha_beta(state, other_player, max_player, alpha, beta)

        state.board[possible_move] = ' '
        state.current_winner = None
        sim_score['position'] = possible_move

        if player == max_player:
            if sim_score['score'] > best['score']:
                best = sim_score
            alpha = max(alpha, sim_score['score'])
        else:
            if sim_score['score'] < best['score']:
                best = sim_score
            beta = min(beta, sim_score['score'])

        # Pruning
        if beta <= alpha:
            break

    return best

In [5]:
def minimax(state, player, maximizing_player):
    max_player = maximizing_player  # 'X' or 'O'
    other_player = 'O' if player == 'X' else 'X'

    # Base cases
    if state.current_winner == other_player:
        return {'position': None, 'score': 1 * (len(state.available_moves()) + 1) if other_player == max_player else -1 * (len(state.available_moves()) + 1)}
    elif not state.empty_squares():
        return {'position': None, 'score': 0}

    if player == max_player:
        best = {'position': None, 'score': -float('inf')}
    else:
        best = {'position': None, 'score': float('inf')}

    for possible_move in state.available_moves():
        # Try move
        state.make_move(possible_move, player)
        sim_score = minimax(state, other_player, max_player)

        # Undo move
        state.board[possible_move] = ' '
        state.current_winner = None
        sim_score['position'] = possible_move

        # Update best
        if player == max_player:
            if sim_score['score'] > best['score']:
                best = sim_score
        else:
            if sim_score['score'] < best['score']:
                best = sim_score

    return best


In [13]:
import time

def benchmark_algorithms():
    game = TicTacToe()

    print("\nBenchmarking Minimax...")
    start_time = time.time()
    minimax(game, 'X', 'X')
    minimax_time = time.time() - start_time
    print(f"Minimax Time: {minimax_time:.6f} seconds")

    game = TicTacToe()  # New fresh board again

    print("\nBenchmarking Minimax with Alpha-Beta Pruning...")
    start_time = time.time()
    minimax_alpha_beta(game, 'X', 'X')
    alpha_beta_time = time.time() - start_time
    print(f"Minimax with Alpha-Beta Pruning Time: {alpha_beta_time:.6f} seconds")

    # Compare Results
    if alpha_beta_time < minimax_time:
        print(f"\n Alpha-Beta Pruning is faster by {minimax_time - alpha_beta_time:.6f} seconds!")
    else:
        print(f"\n No significant speedup noticed (small board size like Tic-Tac-Toe).")


In [15]:
if __name__ == '__main__':
    benchmark_algorithms()



Benchmarking Minimax...
Minimax Time: 6.445336 seconds

Benchmarking Minimax with Alpha-Beta Pruning...
Minimax with Alpha-Beta Pruning Time: 0.236682 seconds

✅ Alpha-Beta Pruning is faster by 6.208654 seconds!
